In [3]:
import time, subprocess, os
import concurrent.futures
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException, NoSuchElementException
from utils import clear_sessions, check_and_run_docker, get_proxies, check_container

In [4]:
# country_css = "div.firm-container > div.firm-services > div.firm-location.custom_tooltip > span::text"
# country = None
# company_block_css = "#service-providers > div > div.directory-list > ul > li.firm-wrapper"
# company_name_css = "div.firm-header > div > h3 > a::text"
# website_link_css = "div.firm-header > a::attr(href)"
# next_page_css = "#service-providers > div > div.directory-list > div.pagination-wrapper > ul > li.next-page > a"
# listing_website_start_url = "https://www.goodfirms.co/it-services/australia"
# page_num = 161
# chromedriver_command_executor = "http://localhost:4444/wd/hub"
# folder_name = "goodfirms_au"
# is_next_page_url_in_format = False
# category = "IT Services"
# last_page = None

In [321]:
country = None
country_css = "div > div.agency-item--inner > div.agency-item--board > span:nth-child(1) > address *::text"
company_block_css = "#js-listing-app > ul > li.agency-list-item.agency-item"
company_name_css = "div > div.agency-item--inner > div.agency-item--main > div > div.info-head-wrap > div > h3 > a::text"
website_link_css = "div > div.agency-item--actions > a::attr(href)"
next_page_css = "#paginator > ul > li.page-item.active+li>a"
is_next_page_url_in_format = True
chromedriver_command_executor = "http://localhost:4444/wd/hub"
folder_name = "designrush_au"
listing_website_start_url = 'https://www.designrush.com/agency/digital-agencies/au'
last_page_css = "#paginator > ul > li:last-child > a"
page_num = 1
last_page = None
category = 'Digital Agencies'
# category_list = {'Branding Agencies': 'https://www.designrush.com/agency/logo-branding/au', 'Creative Agencies': 'https://www.designrush.com/agency/creative-agencies/au', 'Product Design Companies': 'https://www.designrush.com/agency/product-design/au', 'Logo Design Companies': 'https://www.designrush.com/agency/logo-design/au', 'Graphic Design Companies': 'https://www.designrush.com/agency/graphic-design/au', 'Packaging Design Companies': 'https://www.designrush.com/agency/package-design/au', 'Video Production Companies': 'https://www.designrush.com/agency/video-production/au', 'Public Relations Firms': 'https://www.designrush.com/agency/public-relations/au', 'Design Agencies': 'https://www.designrush.com/agency/design-agencies/au', 'Reputation Management Companies': 'https://www.designrush.com/agency/reputation-management-companies/au', 'Web Design Companies': 'https://www.designrush.com/agency/website-design-development/au', 'eCommerce Development Companies': 'https://www.designrush.com/agency/ecommerce/au', 'Web Development Companies': 'https://www.designrush.com/agency/web-development-companies/au', 'WordPress Web Design Companies': 'https://www.designrush.com/agency/wordpress-web-design/au', 'WordPress Development Companies': 'https://www.designrush.com/agency/web-development-companies/wordpress/au', 'Magento Development Companies': 'https://www.designrush.com/agency/ecommerce/magento/au', 'Shopify Development Companies': 'https://www.designrush.com/agency/shopify/au', 'UI/UX Design Agencies': 'https://www.designrush.com/agency/ui-ux-design/au', 'Small Business Website Design Companies': 'https://www.designrush.com/agency/website-design-development/small-business/au', 'Digital Marketing Agencies': 'https://www.designrush.com/agency/digital-marketing/au', 'SEO Agencies': 'https://www.designrush.com/agency/search-engine-optimization/au', 'PPC Agencies': 'https://www.designrush.com/agency/paid-media-pay-per-click/au', 'Social Media Marketing Agencies': 'https://www.designrush.com/agency/social-media-marketing/au', 'Search Engine Marketing Agencies': 'https://www.designrush.com/agency/digital-marketing/search-engine/au', 'Email Marketing Agencies': 'https://www.designrush.com/agency/email-marketing/au', 'Small Business SEO Companies': 'https://www.designrush.com/agency/search-engine-optimization/small-business/au', 'Local SEO Companies': 'https://www.designrush.com/agency/search-engine-optimization/local-seo/au', 'Google Ads Agencies': 'https://www.designrush.com/agency/paid-media-pay-per-click/google-adwords/au', 'Advertising Agencies': 'https://www.designrush.com/agency/ad-agencies/au', 'eCommerce SEO Agencies': 'https://www.designrush.com/agency/search-engine-optimization/ecommerce-seo/au', 'Media Buying Agencies': 'https://www.designrush.com/agency/media-buying-agencies/au', 'Content Marketing Agencies': 'https://www.designrush.com/agency/content-marketing/au', 'Software Development Companies': 'https://www.designrush.com/agency/software-development/au', 'Offshore Software Development Companies': 'https://www.designrush.com/agency/software-development/offshore-software-developers/au', 'Outsourcing Software Development Companies': 'https://www.designrush.com/agency/outsourcing-companies/au', 'Mobile App Development Companies': 'https://www.designrush.com/agency/mobile-app-design-development/au', 'VR &amp; Augmented Reality Companies': 'https://www.designrush.com/agency/ar-vr/au', 'AI Companies': 'https://www.designrush.com/agency/ai-companies/au', 'Android App Development Companies': 'https://www.designrush.com/agency/mobile-app-design-development/android/au', 'iPhone App Development Companies': 'https://www.designrush.com/agency/mobile-app-design-development/iphone/au', 'Blockchain Development Companies': 'https://www.designrush.com/agency/blockchain-development-companies/au', 'Software Testing Companies': 'https://www.designrush.com/agency/software-testing-companies/au', 'IT Services Companies': 'https://www.designrush.com/agency/it-services/au', 'IT Outsourcing Companies': 'https://www.designrush.com/agency/it-services/outsourcing/au', 'Managed Service Providers': 'https://www.designrush.com/agency/managed-service-providers/au', 'Cybersecurity Companies': 'https://www.designrush.com/agency/cybersecurity/au', 'Big Data Analytics Companies': 'https://www.designrush.com/agency/big-data-analytics-companies/au', 'Cloud Consulting Companies': 'https://www.designrush.com/agency/cloud-consulting/au', 'Staff Augmentation Services': 'https://www.designrush.com/agency/staff-augmentation/au', 'SharePoint Consultants': 'https://www.designrush.com/agency/it-services/microsoft-sharepoint/au'}
category_list = {'Branding Agencies': 'https://www.designrush.com/agency/logo-branding/au', 'Creative Agencies': 'https://www.designrush.com/agency/creative-agencies/au', 'Product Design Companies': 'https://www.designrush.com/agency/product-design/au', 'Logo Design Companies': 'https://www.designrush.com/agency/logo-design/au', 'Graphic Design Companies': 'https://www.designrush.com/agency/graphic-design/au', 'Packaging Design Companies': 'https://www.designrush.com/agency/package-design/au', 'Video Production Companies': 'https://www.designrush.com/agency/video-production/au', 'Public Relations Firms': 'https://www.designrush.com/agency/public-relations/au', 'Design Agencies': 'https://www.designrush.com/agency/design-agencies/au', 'Reputation Management Companies': 'https://www.designrush.com/agency/reputation-management-companies/au', 'Web Design Companies': 'https://www.designrush.com/agency/website-design-development/au', 'eCommerce Development Companies': 'https://www.designrush.com/agency/ecommerce/au', 'Web Development Companies': 'https://www.designrush.com/agency/web-development-companies/au', 'WordPress Web Design Companies': 'https://www.designrush.com/agency/wordpress-web-design/au', 'WordPress Development Companies': 'https://www.designrush.com/agency/web-development-companies/wordpress/au', 'Magento Development Companies': 'https://www.designrush.com/agency/ecommerce/magento/au', 'Shopify Development Companies': 'https://www.designrush.com/agency/shopify/au','UIUX Design Agencies': 'https://www.designrush.com/agency/ui-ux-design/au' }

In [322]:
# country_css = "div.provider-card__body.provider-body > ul > li:nth-child(3) > span > span::text"
# country = None
# company_block_css = ".directory-providers__list >li.provider-card"
# company_name_css = "div.provider-header > h3 > a::attr(title)"
# website_link_css = "div.provider-card__body.provider-body > div.provider-card__cta > a.provider-card__visit-btn.provider-visit.track-website-visit::attr(href)"
# next_page_css = "#directory-providers > div > nav > ul > li.page-item.next > a"
# listing_website_start_url = "https://themanifest.com/au/software-testing/companies"
# page_num = 901
# chromedriver_command_executor = "http://localhost:4444/wd/hub"
# folder_name = "themanifest_au"
# is_next_page_url_in_format = False
# last_page = None

In [323]:
if not os.path.exists('data'):
    os.makedirs('data')
if not os.path.exists(f"data/{folder_name}"):
    os.makedirs(f"data/{folder_name}")
    os.makedirs(f"data/{folder_name}/emails")
    os.makedirs(f"data/{folder_name}/logs")
    os.makedirs(f"data/{folder_name}/out")

In [324]:
def run_crawl(full_path, page_num, country=country, country_css=country_css, company_block_css=company_block_css, company_name_css=company_name_css, website_link_css=website_link_css, category=category):
    subprocess.Popen(f'''cd {os.getcwd()} && \
                    scrapy crawl fetch_emails \
                    -a start_url="file://{full_path}" \
                    {f'-a country="{country}"' if country else ''} \
                    {f'-a country_css="{country_css}"' if country_css else ''} \
                    -a company_block_css="{company_block_css}" \
                    -a company_name_css="{company_name_css}" \
                    -a website_link_css="{website_link_css}" \
                    -a category="{category}" \
                    -o "data/{folder_name}/emails/{folder_name} {category} {country if country else ""}{page_num}.csv" \
                    --logfile "data/{folder_name}/logs/{folder_name} {category} {country if country else ""}{page_num}.log"''',
                    shell=True)

In [325]:
# def get_companies(url_to_scrape=listing_website_start_url, page_num=page_num):
def get_companies(url_to_scrape=listing_website_start_url):

    global last_page, category_list, category, page_num
    print(category_list)
    # Set the options for the webdriver
    # PROXIES = get_proxies()
    options = Options()
    next_page = url_to_scrape
    driver = None
    try:
        signal = True
        options.add_argument('--ignore-certificate-errors')
        options.add_argument('--ignore-ssl-errors=yes')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        # options.add_argument("--headless")

        # Set the proxy
        # random_proxy = random.choice(PROXIES)
        # print("Proxy in use: %s" % random_proxy)
        # options.add_argument('--proxy-server=%s' % random_proxy)
        # options.add_argument('--proxy-server=%s' % '104.234.157.100:21278')

        driver = webdriver.Remote(
            command_executor=chromedriver_command_executor,
            options=options,
        )
        print(last_page)
        with concurrent.futures.ProcessPoolExecutor(max_workers=10) as executor:
            next_page_url = listing_website_start_url
            while next_page and (not last_page or page_num <= last_page):
                print(next_page)
                driver.get(next_page)
                print(driver.title)
                if driver.current_url != next_page or driver.title == "404":
                    last_page = None
                    category, next_page_url = category_list.popitem()
                    next_page = next_page_url
                    page_num = 1
                    continue
                elif driver.title == "Just a moment...":
                    print("Just a moment...")
                    print("Waiting for 60 seconds")
                    time.sleep(60)
                    continue
                # elif driver.title == "404":
                   
                file_path = f'data/{folder_name}/out/{folder_name} {category} {page_num}.html'
                full_path = os.path.abspath(file_path)
                

                with open(file_path, 'w') as f:
                    f.write(driver.page_source)
                executor.submit(run_crawl, full_path, page_num, category=category)
                if is_next_page_url_in_format:
                    next_page = next_page_url + f"?page={page_num+1}"
                else:
                    next_page_element = driver.find_element(By.CSS_SELECTOR, next_page_css)
                    if next_page_element:
                        next_page = next_page_element.get_attribute('href')
                    else:
                        next_page = None
                    print(next_page)
                # time.sleep(5)






                if not last_page:
                    try:
                        last_page_element = driver.find_element(By.CSS_SELECTOR, last_page_css)
                    except NoSuchElementException:
                        last_page_element = None
                        category, next_page_url = category_list.popitem()
                        next_page = next_page_url
                        page_num = 0


                    if last_page_element:
                        last_page = int(last_page_element.text)
                elif last_page==page_num:
                    last_page = None
                    category, next_page_url = category_list.popitem()
                    next_page = next_page_url
                    page_num = 0
                page_num += 1

    
    except NoSuchElementException as e:
        print("No Such Element Exception")
        print(e)
        signal = False
    except WebDriverException as e:
        signal = (page_num, next_page)
        print(e)
    except Exception as e:
        print(e)
        print(e.__class__.__name__)
        signal = False
    finally:
        if driver:
            driver.close()
        clear_sessions()
        return signal


In [326]:
# check_and_run_docker()
# time.sleep(10)
# clear_sessions()

In [327]:
clear_sessions()
signal = get_companies(listing_website_start_url)
print(signal)
while isinstance(signal, tuple):
    # signal = get_companies(signal[1],signal[0])
    signal = get_companies(signal[1])
    print(signal)
    time.sleep(10)

{'Branding Agencies': 'https://www.designrush.com/agency/logo-branding/au', 'Creative Agencies': 'https://www.designrush.com/agency/creative-agencies/au', 'Product Design Companies': 'https://www.designrush.com/agency/product-design/au', 'Logo Design Companies': 'https://www.designrush.com/agency/logo-design/au', 'Graphic Design Companies': 'https://www.designrush.com/agency/graphic-design/au', 'Packaging Design Companies': 'https://www.designrush.com/agency/package-design/au', 'Video Production Companies': 'https://www.designrush.com/agency/video-production/au', 'Public Relations Firms': 'https://www.designrush.com/agency/public-relations/au', 'Design Agencies': 'https://www.designrush.com/agency/design-agencies/au', 'Reputation Management Companies': 'https://www.designrush.com/agency/reputation-management-companies/au', 'Web Design Companies': 'https://www.designrush.com/agency/website-design-development/au', 'eCommerce Development Companies': 'https://www.designrush.com/agency/ecom

In [318]:
print(category_list)

{'Branding Agencies': 'https://www.designrush.com/agency/logo-branding/au', 'Creative Agencies': 'https://www.designrush.com/agency/creative-agencies/au', 'Product Design Companies': 'https://www.designrush.com/agency/product-design/au', 'Logo Design Companies': 'https://www.designrush.com/agency/logo-design/au', 'Graphic Design Companies': 'https://www.designrush.com/agency/graphic-design/au', 'Packaging Design Companies': 'https://www.designrush.com/agency/package-design/au', 'Video Production Companies': 'https://www.designrush.com/agency/video-production/au', 'Public Relations Firms': 'https://www.designrush.com/agency/public-relations/au', 'Design Agencies': 'https://www.designrush.com/agency/design-agencies/au', 'Reputation Management Companies': 'https://www.designrush.com/agency/reputation-management-companies/au', 'Web Design Companies': 'https://www.designrush.com/agency/website-design-development/au', 'eCommerce Development Companies': 'https://www.designrush.com/agency/ecom

In [320]:
category_list

{'Branding Agencies': 'https://www.designrush.com/agency/logo-branding/au',
 'Creative Agencies': 'https://www.designrush.com/agency/creative-agencies/au',
 'Product Design Companies': 'https://www.designrush.com/agency/product-design/au',
 'Logo Design Companies': 'https://www.designrush.com/agency/logo-design/au',
 'Graphic Design Companies': 'https://www.designrush.com/agency/graphic-design/au',
 'Packaging Design Companies': 'https://www.designrush.com/agency/package-design/au',
 'Video Production Companies': 'https://www.designrush.com/agency/video-production/au',
 'Public Relations Firms': 'https://www.designrush.com/agency/public-relations/au',
 'Design Agencies': 'https://www.designrush.com/agency/design-agencies/au',
 'Reputation Management Companies': 'https://www.designrush.com/agency/reputation-management-companies/au',
 'Web Design Companies': 'https://www.designrush.com/agency/website-design-development/au',
 'eCommerce Development Companies': 'https://www.designrush.com/